In [7]:
import heapq
import random
import math
import re
import itertools
import time
import threading

class TimeoutException(Exception):
    """Exception raised when a function times out."""
    pass

class VertexOrderingProblem:
    def __init__(self, filename):
        """
        Initialize the problem with data from the given file.
        
        Args:
            filename: Path to the data file
        """
        self.vertices = set()
        self.parent_sets = {}  # vertex -> [(parent_set, cost), ...]
        self.parse_file(filename)
        
    def parse_file(self, filename):
        """
        Parse the input file and populate the data structures.
        
        Args:
            filename: Path to the data file
        """
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                match = re.match(r"(\d+),\{([^}]*)\},([0-9.]+)", line)
                if not match:
                    continue
                    
                vertex = int(match.group(1))
                parent_set_str = match.group(2)
                cost = float(match.group(3))
                
                # Convert parent set string to tuple of integers
                if parent_set_str:
                    parent_set = tuple(sorted(map(int, parent_set_str.split(","))))
                else:
                    parent_set = tuple()
                
                self.vertices.add(vertex)
                
                if vertex not in self.parent_sets:
                    self.parent_sets[vertex] = []
                self.parent_sets[vertex].append((parent_set, cost))
    
    def is_consistent(self, parent_set, ordering, vertex_idx):
        """
        Check if a parent set is consistent with an ordering.
        
        Args:
            parent_set: Tuple of parent vertices
            ordering: List representing an ordering of vertices
            vertex_idx: Index of current vertex in ordering
            
        Returns:
            bool: True if parent set is consistent with ordering
        """
        vertex_position = vertex_idx
        # Check if all parents come before the vertex in the ordering
        return all(ordering.index(parent) < vertex_position for parent in parent_set)
    
    def min_consistent_cost(self, vertex, ordering, vertex_idx):
        """
        Get minimum cost of parent sets for a vertex that are consistent with an ordering.
        
        Args:
            vertex: The vertex to find parent sets for
            ordering: List representing an ordering of vertices
            vertex_idx: Index of current vertex in ordering
            
        Returns:
            float: Minimum cost of consistent parent sets, or infinity if none exist
        """
        min_cost = float('inf')
        for parent_set, cost in self.parent_sets[vertex]:
            # Check if all parents come before vertex in ordering
            if all(parent in ordering[:vertex_idx] for parent in parent_set):
                min_cost = min(min_cost, cost)
        return min_cost
    
    def total_cost(self, ordering):
        """
        Calculate total cost of an ordering.
        
        Args:
            ordering: List representing an ordering of vertices
            
        Returns:
            float: Total cost of the ordering
        """
        total = 0
        for i, vertex in enumerate(ordering):
            cost = self.min_consistent_cost(vertex, ordering, i)
            if cost == float('inf'):
                return float('inf')
            total += cost
        return total
    
    def brute_force_search(self, timeout=300):
        """
        Try all possible orderings and return the best one.
        
        Args:
            timeout: Maximum time in seconds before timing out
            
        Returns:
            tuple: (best_ordering, best_cost)
        """
        best_ordering = None
        best_cost = float('inf')
        
        start_time = time.time()
        
        try:
            for ordering in itertools.permutations(self.vertices):
                # Check for timeout
                if time.time() - start_time > timeout:
                    print(f"Brute force search timed out after {timeout} seconds")
                    break
                    
                cost = self.total_cost(ordering)
                if cost < best_cost:
                    best_cost = cost
                    best_ordering = ordering
        except Exception as e:
            print(f"Error during brute force search: {e}")
        
        return best_ordering, best_cost
    
    def hill_climbing(self, max_iterations=1000, timeout=300):
        """
        Hill climbing search for the best ordering.
        
        Args:
            max_iterations: Maximum number of iterations
            timeout: Maximum time in seconds before timing out
            
        Returns:
            tuple: (best_ordering, best_cost)
        """
        # Start with a random ordering
        current_ordering = list(self.vertices)
        random.shuffle(current_ordering)
        current_cost = self.total_cost(current_ordering)
        
        start_time = time.time()
        
        try:
            for _ in range(max_iterations):
                # Check if we should exit due to timeout
                if time.time() - start_time > timeout:
                    print(f"Hill climbing search timed out after {timeout} seconds")
                    break
                
                # Try swapping pairs of vertices
                improved = False
                for i in range(len(current_ordering)):
                    for j in range(i+1, len(current_ordering)):
                        # Check for timeout between swaps
                        if time.time() - start_time > timeout:
                            print(f"Hill climbing search timed out after {timeout} seconds")
                            break
                            
                        # Create a new ordering by swapping
                        new_ordering = current_ordering.copy()
                        new_ordering[i], new_ordering[j] = new_ordering[j], new_ordering[i]
                        
                        # Calculate new cost
                        new_cost = self.total_cost(new_ordering)
                        
                        # If better, update current ordering
                        if new_cost < current_cost:
                            current_ordering = new_ordering
                            current_cost = new_cost
                            improved = True
                            break
                    
                    if improved or time.time() - start_time > timeout:
                        break
                
                # If no improvement, we're at a local optimum
                if not improved:
                    break
        except Exception as e:
            print(f"Error during hill climbing search: {e}")
        
        return tuple(current_ordering), current_cost
    
    def simulated_annealing(self, initial_temp=100.0, cooling_rate=0.95, max_iterations=1000, timeout=300):
        """
        Simulated annealing search for the best ordering.
        
        Args:
            initial_temp: Initial temperature
            cooling_rate: Cooling rate
            max_iterations: Maximum number of iterations
            timeout: Maximum time in seconds before timing out
            
        Returns:
            tuple: (best_ordering, best_cost)
        """
        # Start with a random ordering
        current_ordering = list(self.vertices)
        random.shuffle(current_ordering)
        current_cost = self.total_cost(current_ordering)
        
        best_ordering = current_ordering.copy()
        best_cost = current_cost
        
        temp = initial_temp
        
        start_time = time.time()
        
        try:
            for iteration in range(max_iterations):
                # Check if we should exit (either due to timeout or temperature too low)
                if time.time() - start_time > timeout or temp < 0.1:
                    if time.time() - start_time > timeout:
                        print(f"Simulated annealing search timed out after {timeout} seconds")
                    break
                
                # Pick random indices to swap
                i, j = random.sample(range(len(current_ordering)), 2)
                
                # Create a new ordering by swapping
                new_ordering = current_ordering.copy()
                new_ordering[i], new_ordering[j] = new_ordering[j], new_ordering[i]
                
                # Calculate new cost
                new_cost = self.total_cost(new_ordering)
                
                # Decide whether to accept the new ordering
                if new_cost < current_cost:
                    # Always accept better solutions
                    delta = 0
                else:
                    delta = new_cost - current_cost
                
                # Accept with probability e^(-delta/temp)
                if delta == 0 or random.random() < math.exp(-delta / temp):
                    current_ordering = new_ordering
                    current_cost = new_cost
                    
                    # Update best if needed
                    if current_cost < best_cost:
                        best_ordering = current_ordering.copy()
                        best_cost = current_cost
                
                # Cool the temperature
                temp *= cooling_rate
        except Exception as e:
            print(f"Error during simulated annealing search: {e}")
        
        return tuple(best_ordering), best_cost
    
    def a_star_search(self, max_states=100000, timeout=300):
        """
        A* search for the best ordering.
        
        Args:
            max_states: Maximum number of states to explore
            timeout: Maximum time in seconds before timing out
            
        Returns:
            tuple: (best_ordering, best_cost)
        """
        # Priority queue will store (f, g, ordering, explored)
        # where f = g + h (cost so far + heuristic), g = cost so far
        # and explored is a set of vertices already in the ordering
        start_state = (0, 0, (), frozenset())
        priority_queue = [start_state]
        visited = set()
        
        start_time = time.time()
        
        try:
            states_explored = 0
            
            while priority_queue and states_explored < max_states:
                # Check if we should exit due to timeout
                if time.time() - start_time > timeout:
                    print(f"A* search timed out after {timeout} seconds")
                    break
                
                states_explored += 1
                f, g, ordering, explored = heapq.heappop(priority_queue)
                
                if frozenset(ordering) in visited:
                    continue
                
                visited.add(frozenset(ordering))
                
                # If we have a complete ordering, return it
                if len(ordering) == len(self.vertices):
                    return ordering, g
                
                # Try adding each unexplored vertex next
                for next_vertex in self.vertices:
                    # Check for timeout periodically
                    if states_explored % 100 == 0 and time.time() - start_time > timeout:
                        print(f"A* search timed out after {timeout} seconds")
                        break
                        
                    if next_vertex not in explored:
                        new_ordering = ordering + (next_vertex,)
                        new_explored = explored.union({next_vertex})
                        
                        # Calculate cost of adding this vertex with current ordering
                        cost_to_add = self.min_consistent_cost(next_vertex, new_ordering, len(new_ordering) - 1)
                        if cost_to_add == float('inf'):
                            continue
                        
                        new_g = g + cost_to_add
                        
                        # Simple heuristic: estimate remaining cost as 0 (optimistic)
                        # Can be improved with more domain knowledge
                        new_f = new_g
                        
                        heapq.heappush(priority_queue, (new_f, new_g, new_ordering, new_explored))
        except Exception as e:
            print(f"Error during A* search: {e}")
        
        # If no solution found or timeout, return the best partial ordering we have
        if priority_queue:
            best_state = min(priority_queue, key=lambda x: x[0])
            return best_state[2], float('inf')
        return None, float('inf')

In [11]:
def main():
    filename = r"C:\Users\hp\Downloads\data3.txt"
    
    print(f"Testing with file: {filename}")
    
    # Initialize the problem
    problem = VertexOrderingProblem(filename)
    print(f"Vertices: {problem.vertices}")
    print(f"Number of vertices: {len(problem.vertices)}")
    
    # Adaptive timeout based on problem size
    timeout = min(300, max(30, 10 * len(problem.vertices)))
    print(f"Using timeout of {timeout} seconds per algorithm")
    
    # If the problem is small enough, try brute force
    if len(problem.vertices) <= 10:
        print("\nRunning Brute Force search...")
        start_time = time.time()
        best_ordering, best_cost = problem.brute_force_search(timeout=timeout)
        elapsed = time.time() - start_time
        if best_ordering:
            print(f"Best ordering: {best_ordering}")
            print(f"Cost: {best_cost}")
        else:
            print("No solution found within time limit")
        print(f"Time elapsed: {elapsed:.4f} seconds")
    else:
        print("\nSkipping Brute Force search (too many vertices)")
    
    # Hill Climbing
    print("\nRunning Hill Climbing search...")
    start_time = time.time()
    best_ordering, best_cost = problem.hill_climbing(timeout=timeout)
    elapsed = time.time() - start_time
    if best_ordering:
        print(f"Best ordering: {best_ordering}")
        print(f"Cost: {best_cost}")
    else:
        print("No solution found within time limit")
    print(f"Time elapsed: {elapsed:.4f} seconds")
    
    # Simulated Annealing
    print("\nRunning Simulated Annealing search...")
    start_time = time.time()
    best_ordering, best_cost = problem.simulated_annealing(timeout=timeout)
    elapsed = time.time() - start_time
    if best_ordering:
        print(f"Best ordering: {best_ordering}")
        print(f"Cost: {best_cost}")
    else:
        print("No solution found within time limit")
    print(f"Time elapsed: {elapsed:.4f} seconds")
    
    # A* Search (for smaller problems)
    if len(problem.vertices) <= 15:
        print("\nRunning A* search...")
        start_time = time.time()
        best_ordering, best_cost = problem.a_star_search(timeout=timeout)
        elapsed = time.time() - start_time
        if best_ordering and best_cost != float('inf'):
            print(f"Best ordering: {best_ordering}")
            print(f"Cost: {best_cost}")
        else:
            print("No complete solution found within time limit")
            if best_ordering:
                print(f"Partial ordering: {best_ordering}")
        print(f"Time elapsed: {elapsed:.4f} seconds")
    else:
        print("\nSkipping A* search (too many vertices)")
    
    # Print overall best solution
    print("\nOverall best solution:")
    solutions = []
    algorithms = []
    
    if len(problem.vertices) <= 10:
        bf_ordering, bf_cost = problem.brute_force_search(timeout=timeout)
        algorithms.append("Brute Force")
        solutions.append((bf_ordering, bf_cost))
    
    hc_ordering, hc_cost = problem.hill_climbing(timeout=timeout)
    sa_ordering, sa_cost = problem.simulated_annealing(timeout=timeout)
    
    algorithms.extend(["Hill Climbing", "Simulated Annealing"])
    solutions.extend([
        (hc_ordering, hc_cost),
        (sa_ordering, sa_cost)
    ])
    
    if len(problem.vertices) <= 15:
        algorithms.append("A*")
        a_star_result = problem.a_star_search(timeout=timeout)
        if a_star_result[1] != float('inf'):
            solutions.append(a_star_result)
    
    # Find the best solution
    valid_solutions = [(o, c) for o, c in solutions if o is not None and c != float('inf')]
    if valid_solutions:
        best_solution = min(valid_solutions, key=lambda x: x[1])
        best_algorithm = algorithms[solutions.index(best_solution)]
        
        print(f"Best algorithm: {best_algorithm}")
        print(f"Best ordering: {best_solution[0]}")
        print(f"Cost: {best_solution[1]}")
    else:
        print("No valid solutions found")

if __name__ == "__main__":
    main()

Testing with file: C:\Users\hp\Downloads\data3.txt
Vertices: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
Number of vertices: 19
Using timeout of 190 seconds per algorithm

Skipping Brute Force search (too many vertices)

Running Hill Climbing search...
Best ordering: (8, 19, 2, 18, 12, 9, 1, 10, 7, 4, 16, 15, 14, 5, 11, 6, 13, 3, 17)
Cost: 7996.777999999998
Time elapsed: 0.5163 seconds

Running Simulated Annealing search...
Best ordering: (12, 11, 9, 7, 18, 14, 8, 5, 1, 15, 2, 3, 17, 4, 10, 19, 6, 16, 13)
Cost: 8009.905
Time elapsed: 0.0871 seconds

Skipping A* search (too many vertices)

Overall best solution:
Best algorithm: Simulated Annealing
Best ordering: (12, 5, 11, 8, 3, 17, 2, 10, 13, 18, 19, 9, 7, 4, 6, 1, 15, 14, 16)
Cost: 8001.136999999999
